# BROCHURE BUILDER

In [ ]:
# pip install requests dotenv beautifulSoup4 openai

In [ ]:
import os
import requests
from openai import OpenAI
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [ ]:
# Initialize the constants

load_dotenv(override=True)
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

if OPENAI_KEY and OPENAI_KEY.startswith("sk-proj-") and len(OPENAI_KEY) > 10:
    print("The api key looks good")
else:
    print("There might be a problem with the api key")

MODEL = 'gpt-4o-mini'
openai = OpenAI()



In [ ]:
# Class which represents webpage
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body():
            for irrelevant in soup.body(['script', 'img', 'input', 'style']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else :   
            self.text = ''
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        

# website = Website("https://www.cypherock.com/?srsltid=AfmBOooDbHMbrl25kt6F4EwMY8zR_Oit2VuZr-GODuBv5KasLaBdqUn9")
# website.links
# website.get_contents()
        

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

# print(link_system_prompt)

In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
                Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

# print(get_links_user_prompt(website))

In [ ]:
def get_links(website):
    website = Website("https://www.cypherock.com/?srsltid=AfmBOooDbHMbrl25kt6F4EwMY8zR_Oit2VuZr-GODuBv5KasLaBdqUn9")
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            { 'role':'system', 'content':link_system_prompt},
            {'role':'user', 'content':get_links_user_prompt(website)}
        ],
        response_format={'type':'json_object'}
    )

    result = response.choices[0].message.content
    return json.loads(result)

# get_links(website)

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

# get_brochure_user_prompt('CypherRock', "https://www.cypherock.com/?srsltid=AfmBOooDbHMbrl25kt6F4EwMY8zR_Oit2VuZr-GODuBv5KasLaBdqUn9")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system', 'content':system_prompt},
            {'role':'user', 'content':get_brochure_user_prompt(company_name, url)}
        ]
    )

    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure('CypherRock', "https://www.cypherock.com/?srsltid=AfmBOooDbHMbrl25kt6F4EwMY8zR_Oit2VuZr-GODuBv5KasLaBdqUn9")